In [25]:
import sys
import os

path0 = os.path.abspath(os.path.join('.'))
path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path)
    sys.path.append(path0)
path, path0

('/home/kenji/Fyp/DeepMimic_mujoco',
 '/home/kenji/Fyp/DeepMimic_mujoco/diffusion')

In [26]:
from diffusion.data_loaders.report_dataset import SpinkickPureFramesDataset
dataset = SpinkickPureFramesDataset("/home/kenji/Fyp/DeepMimic_mujoco/diffusion/data/motions/humanoid3d_spinkick.txt")
len(dataset), dataset.motion_data.shape

torch.Size([78, 44])


(16, torch.Size([16, 78, 44]))

In [27]:
dataset[0]

tensor([[ 0.0167,  0.0000,  0.8251,  ..., -0.3155,  0.2235,  1.4452],
        [ 0.0167, -0.0098,  0.8223,  ..., -0.2932,  0.2308,  1.4881],
        [ 0.0167, -0.0205,  0.8187,  ..., -0.2659,  0.2363,  1.5274],
        ...,
        [ 0.0167,  0.4499,  0.7884,  ..., -0.3115,  0.1044,  1.2661],
        [ 0.0167,  0.4598,  0.8026,  ..., -0.2961,  0.0695,  1.1802],
        [ 0.0000,  0.4689,  0.8251,  ..., -0.2681,  0.0327,  1.1281]])

### Model NN

In [28]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=1000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)

        self.register_buffer('pe', pe)

    def forward(self, x):
        # not used in the final model
        x = x + self.pe[:x.shape[0], :]
        return self.dropout(x)


class TimestepEmbedder(nn.Module):
    def __init__(self, latent_dim, sequence_pos_encoder):
        super().__init__()
        self.latent_dim = latent_dim
        self.sequence_pos_encoder = sequence_pos_encoder

        time_embed_dim = self.latent_dim
        self.time_embed = nn.Sequential(
            nn.Linear(self.latent_dim, time_embed_dim),
            nn.SiLU(),
            nn.Linear(time_embed_dim, time_embed_dim),
        )

    def forward(self, timesteps):
        return self.time_embed(self.sequence_pos_encoder.pe[timesteps])

In [29]:
class MotionTransformer(nn.Module):
    def __init__(self, frame_dim, latent_dim=256, ff_size=1024, num_layers=8, num_heads=4, dropout=0.1, activation="gelu"):
        super(MotionTransformer, self).__init__()
        
        # self.nfeats = nfeats
        self.frame_dim = frame_dim
        self.latent_dim = latent_dim
        self.ff_size = ff_size  
        self.dropout = dropout

        self.frameEmbedding = nn.Linear(self.frame_dim, self.latent_dim)
        self.sequence_pos_encoder = PositionalEncoding(self.latent_dim, self.dropout)
        self.embed_timestep = TimestepEmbedder(self.latent_dim, self.sequence_pos_encoder)

        # Transformer Encoder
        encoder_layers = nn.TransformerEncoderLayer(d_model=self.latent_dim, nhead=num_heads, 
                                                    dim_feedforward=ff_size, dropout=dropout, activation=activation, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

        # Output Linear Layer
        self.frameOutEmbedding = nn.Linear(self.latent_dim, self.frame_dim)

    def forward(self, x: torch.Tensor, timesteps, y=None, verbose=False):
        """
        x: [batch_size, max_frames, n_feats], denoted x_t in the paper
        timesteps: [batch_size] (int)
        """
        # x: [batch_size, seq_len, nfeats]
        emb = self.embed_timestep(timesteps)  # [bs, n_frames, time_embed_dim]
        if verbose:
            print("Emb", emb.shape)
            print(emb)

        x = self.frameEmbedding(x) # [bs, n_frames, n_dim]
        if verbose:
            print("Frame Embedding", x.shape)
            print(x)

        # Transformer Encoder
        # adding the timestep embed
        xseq = torch.cat((emb, x), axis=1)  # [bs, n_frames+1, n_dim]
        if verbose:
            print("Concat x and zkx", xseq.shape)
            print(xseq)

        xseq = self.sequence_pos_encoder(xseq)  # [bs, n_frames+1, n_dim]
        if verbose:
            print("Sequence Pos Encoder", xseq.shape)
            print(xseq)

        output = self.transformer_encoder(xseq)[:, 1:, :]  # , src_key_padding_mask=~maskseq)  # [bs, n_frames, n_dim]
        if verbose:
            print("Transformer Encoder", output.shape)
            print(output)

        # Output Linear
        output = self.frameOutEmbedding(output)
        if verbose:
            print("Output Embedding", output.shape)
            print(output)

        return output

In [30]:
from torch.utils.data import DataLoader

batch_size = 1
dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=True)

frame_dim = dataset[0].shape[1]
duration = dataset[0][0][0]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MotionTransformer(frame_dim=frame_dim, latent_dim=64, ff_size=256, num_layers=8, num_heads=4, dropout=0.1, activation="gelu").to(device)

frame_dim, duration

(44, tensor(0.0167))

In [31]:
test_sample = dataset[0].unsqueeze(0).to(device)    
print(test_sample.shape)

res = model(test_sample, torch.tensor([0]).to(device), verbose=True)
res.shape

torch.Size([1, 78, 44])
Emb torch.Size([1, 1, 64])
tensor([[[-1.4030e-02,  1.7927e-01, -2.1890e-02, -2.4592e-01, -5.0784e-02,
           3.4424e-01,  2.2998e-02, -1.0769e-01,  1.6136e-02, -9.7934e-02,
           2.3989e-01, -2.2320e-02,  1.4624e-02, -3.9569e-02, -3.1032e-01,
          -1.4733e-01, -2.6958e-01, -2.2703e-01, -1.8168e-01, -1.0929e-02,
          -2.2769e-01,  1.8720e-01,  8.2235e-02, -1.2610e-02, -1.7831e-02,
          -1.3792e-01,  1.9889e-02,  3.8692e-02, -1.7119e-01,  5.4054e-02,
          -1.1408e-01,  1.5402e-02,  7.3161e-02,  1.5085e-01,  2.7670e-01,
           1.3350e-01, -9.8427e-02,  1.0871e-01, -5.1510e-02,  1.5777e-01,
           1.2632e-01, -2.9219e-02,  2.3348e-02,  2.3807e-01,  2.3947e-01,
           2.4656e-02, -4.9694e-02, -4.2554e-02, -5.1662e-02,  1.1118e-04,
           1.2233e-01,  9.9781e-02, -1.3964e-01, -2.5684e-01, -3.5022e-02,
          -9.1154e-02,  6.4052e-02,  8.1167e-02, -5.8010e-02,  6.2802e-02,
          -1.3023e-01,  1.6333e-01,  1.3121e-01, 

tensor([[[-0.0140,  0.1793, -0.0219,  ...,  0.1633,  0.1312,  0.0811],
         [ 0.0313,  0.6351, -0.4949,  ...,  0.5134, -0.1462, -0.2112],
         [ 0.0264,  0.6528, -0.4887,  ...,  0.5251, -0.1667, -0.2102],
         ...,
         [ 0.0734,  0.7298, -0.4766,  ...,  0.3209, -0.0980,  0.0803],
         [ 0.0665,  0.7271, -0.4685,  ...,  0.3121, -0.0979,  0.0812],
         [ 0.0591,  0.7329, -0.4597,  ...,  0.3147, -0.0998,  0.0713]]],
       device='cuda:0', grad_fn=<CatBackward0>)
Sequence Pos Encoder torch.Size([1, 79, 64])
tensor([[[-0.0156,  1.3103, -0.0000,  ...,  1.2926,  0.1458,  0.0000],
         [ 0.0348,  1.8167, -0.5499,  ...,  1.6816, -0.1624,  0.8765],
         [ 0.0294,  0.0000, -0.5431,  ...,  1.6945, -0.1853,  0.8776],
         ...,
         [ 0.0815,  1.9220, -0.5295,  ...,  0.0000, -0.1089,  1.2003],
         [ 0.0739,  1.9190, -0.5205,  ...,  1.4578, -0.1088,  1.2014],
         [ 0.0656,  1.9254, -0.0000,  ...,  0.0000, -0.1108,  1.1903]]],
       device='cuda:0',

torch.Size([1, 78, 44])

In [37]:
from diffusion.diffusion import gaussian_diffusion as gd
from diffusion.diffusion.respace import SpacedDiffusion, space_timesteps

def create_gaussian_diffusion(
        diffusion_steps, # number eg 1000
        noise_schedule, # can be 'linear', 'cosine'
        sigma_small, # default True
        lambda_vel, lambda_rcxyz, lambda_fc # for geometric loss, we don't have fc, default 1 for rest
        ):
    # default params
    predict_xstart = True  # we always predict x_start (a.k.a. x0), that's our deal!
    steps = diffusion_steps
    scale_beta = 1.  # no scaling
    timestep_respacing = ''  # can be used for ddim sampling, we don't use it.
    learn_sigma = False
    rescale_timesteps = False

    betas = gd.get_named_beta_schedule(noise_schedule, steps, scale_beta)
    loss_type = gd.LossType.MSE

    if not timestep_respacing:
        timestep_respacing = [steps]

    return SpacedDiffusion(
        use_timesteps=space_timesteps(steps, timestep_respacing),
        betas=betas,
        model_mean_type=(
            gd.ModelMeanType.EPSILON if not predict_xstart else gd.ModelMeanType.START_X
        ),
        model_var_type=(
            (
                gd.ModelVarType.FIXED_LARGE
                if not sigma_small
                else gd.ModelVarType.FIXED_SMALL
            )
            if not learn_sigma
            else gd.ModelVarType.LEARNED_RANGE
        ),
        loss_type=loss_type,
        rescale_timesteps=rescale_timesteps,
        lambda_vel=lambda_vel,
        lambda_rcxyz=lambda_rcxyz,
        lambda_fc=lambda_fc,
    )

In [38]:
model = MotionTransformer(frame_dim=frame_dim, latent_dim=256, ff_size=128, num_layers=8, num_heads=8, dropout=0.1, activation="gelu").to(device)
diffusion = create_gaussian_diffusion(diffusion_steps=1000, noise_schedule="cosine", sigma_small=True, lambda_vel=10, lambda_rcxyz=10, lambda_fc=0)

In [39]:
class DefaultArgs:
    def __init__(self, save_dir, model_path, eval_model_path):
        # Base options
        self.cuda = True
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.seed = 10
        self.batch_size = 1

        # Diffusion options
        self.noise_schedule = 'cosine'
        self.diffusion_steps = 1000
        self.sigma_small = True

        # Model options
        self.arch = 'trans_enc'
        self.emb_trans_dec = False
        self.layers = 8
        self.latent_dim = 128
        self.cond_mask_prob = 0
        self.lambda_rcxyz = 10
        self.lambda_vel = 10
        self.lambda_fc = 0
        self.unconstrained = False  # This is inferred from the 'action' parameter

        # Data options
        self.dataset = ''
        self.data_dir = ""

        # Training options
        self.save_dir = save_dir
        self.overwrite = False
        self.train_platform_type = 'NoPlatform'
        self.lr = 1e-4
        self.weight_decay = 0.0
        self.lr_anneal_steps = 0
        self.eval_batch_size = 16
        self.eval_split = 'test'
        self.eval_during_training = False
        self.eval_rep_times = 3
        self.eval_num_samples = 1000
        self.log_interval = 150
        self.save_interval = 1000
        self.num_steps = 5000
        # self.num_frames = 29
        self.resume_checkpoint = ""

        # Sampling options
        self.model_path = model_path
        self.output_dir = ''
        self.num_samples = 10
        self.num_repetitions = 3
        self.guidance_param = 2.5

        # Generate options
        self.motion_length = 6.0
        self.input_text = ''
        self.action_file = ''
        self.text_prompt = ''
        self.action_name = ''

        # Edit options
        self.edit_mode = 'in_between'
        self.text_condition = ''
        self.prefix_end = 0.25
        self.suffix_start = 0.75

        # Evaluation options
        self.eval_model_path = eval_model_path
        self.eval_mode = 'wo_mm'
        self.eval_guidance_param = 2.5


In [40]:
expname = "report-dissertation-3"

args = DefaultArgs(save_dir=f"/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{expname}", model_path=f"/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{expname}/model.pt", eval_model_path=f"/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{expname}/model.pt")
args.device

'cuda:0'

In [41]:
from train.training_loop import TrainLoop
TrainLoop(args, None , model, diffusion, dataloader).run_loop()

Starting epoch 0/313


  0%|          | 0/16 [00:00<?, ?it/s]

----------------------------
| frame_loss    | 253      |
| frame_loss_q0 | 278      |
| frame_loss_q1 | 215      |
| frame_loss_q2 | 223      |
| frame_loss_q3 | 296      |
| grad_norm     | 1.32e+03 |
| loss          | 1.3e+03  |
| loss_q0       | 1.3e+03  |
| loss_q1       | 1.32e+03 |
| loss_q2       | 1.25e+03 |
| loss_q3       | 1.35e+03 |
| param_norm    | 95.6     |
| rcxyz_mse     | 46.1     |
| rcxyz_mse_q0  | 45.1     |
| rcxyz_mse_q1  | 46.3     |
| rcxyz_mse_q2  | 45.4     |
| rcxyz_mse_q3  | 47.6     |
| samples       | 1        |
| step          | 0        |
| vel_mse       | 59       |
| vel_mse_q0    | 57.2     |
| vel_mse_q1    | 64       |
| vel_mse_q2    | 57.3     |
| vel_mse_q3    | 58.1     |
----------------------------
step[0]: loss[1303.91920]
saving model...


  6%|▋         | 1/16 [00:00<00:08,  1.87it/s]

Skipping evaluation for now.


...

saving model...


 56%|█████▋    | 9/16 [00:03<00:02,  2.66it/s]

Skipping evaluation for now.


100%|██████████| 16/16 [00:05<00:00,  2.84it/s]

saving model...
Skipping evaluation for now.


In [42]:
dataset[0].shape

torch.Size([78, 44])

In [43]:
num_frames = dataset[0].shape[0]
num_feats = dataset[0].shape[1]

In [44]:
args.num_repetitions

3

In [45]:
sample_batch_size = 1

all_motions = []
for rep_i in range(args.num_repetitions):
    print(f'### Sampling [repetitions #{rep_i}]')

    sample_fn = diffusion.p_sample_loop

    sample = sample_fn(
        model,
        (sample_batch_size, num_frames, num_feats),
        clip_denoised=False,
        model_kwargs={"y": {}},
        skip_timesteps=0,  # 0 is the default value - i.e. don't skip any step
        init_image=None,
        progress=True,
        dump_steps=None,
        noise=None,
        const_noise=False,
    )

    all_motions.append(sample.cpu().numpy())

### Sampling [repetitions #0]


100%|██████████| 1000/1000 [00:04<00:00, 212.13it/s]


### Sampling [repetitions #1]


100%|██████████| 1000/1000 [00:05<00:00, 194.53it/s]


### Sampling [repetitions #2]


100%|██████████| 1000/1000 [00:05<00:00, 192.18it/s]


In [46]:
all_motions = np.concatenate(all_motions, axis=0)
all_motions.shape

(3, 78, 44)

In [47]:
dura = 0.0167
for motion in all_motions:
    for frame in motion:
        frame[0] = dura

In [48]:
import numpy as np

def save_motions(all_motions, output_dir):
    for i, motion in enumerate(all_motions):
        filename = f"motion_{i}.npy"
        filepath = os.path.join(output_dir, filename)
        np.save(filepath, motion)
        print(f"Motion {i} saved as {filename}")

save_motions(all_motions, f"/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{expname}/sampled_motions/")

Motion 0 saved as motion_0.npy
Motion 1 saved as motion_1.npy
Motion 2 saved as motion_2.npy
